In [ ]:
!pip install -q evaluate transformers accelerate bitsandbytes peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 886.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from evaluate import load
from datasets import load_from_disk
from peft import PeftModel, PeftConfig
import pandas as pd

In [ ]:
model_name_or_path = "google/flan-t5-large"
tokenizer_name_or_path = "google/flan-t5-large"
peft_model_path = "/content/drive/MyDrive/Colab Notebooks/Table to insights/Flan/prefix_tuned_model"
text_column = "prompt"
label_column = "label"
max_length = 128

In [ ]:
# Specify the directory where the dataset is saved
saved_directory = "/content/drive/MyDrive/Colab Notebooks/Table to insights/Data/Analytical Datset"

# Load the dataset from the specified directory
dataset = load_from_disk(saved_directory)
dataset = dataset['validation']

In [ ]:
dataset

Dataset({
    features: ['table', 'prompt', 'label'],
    num_rows: 129
})

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

def generate_summary(model, tokenizer, query):
    input_ids = tokenizer.encode(query, return_tensors="pt", max_length=max_length,padding="max_length" ,truncation=True)
    summary_ids = model.generate(input_ids, max_length=150, num_beams=4, length_penalty=2.0, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def summarize_example(example, original_model, original_tokenizer, peft_tuned_model, peft_tuned_tokenizer):
    prompt = example[text_column]
    label = example[label_column]

    # Generate summaries using the original model
    original_model_summary = generate_summary(original_model, original_tokenizer, prompt)

    # Generate summaries using the PEFT tuned model
    tuned_summary = generate_summary(peft_tuned_model, peft_tuned_tokenizer, prompt)

    return {"prompt": prompt, "label": label, "original_model_summary": original_model_summary, "tuned_summary": tuned_summary}



In [ ]:
# Load the original model
original_tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
original_model = AutoModelForSeq2SeqLM.from_pretrained(tokenizer_name_or_path,load_in_8bit=True,device_map="auto")

# Load the PEFT tuned model
config = PeftConfig.from_pretrained(peft_model_path)
peft_base_model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
peft_tuned_model = PeftModel.from_pretrained(peft_base_model, peft_model_path)

# Use map to apply the summarization function to each example
summarized_dataset = dataset.map(lambda example: summarize_example(example, original_model, original_tokenizer, peft_tuned_model, original_tokenizer))

# Display or save the results as a DataFrame
df = pd.DataFrame(summarized_dataset)

HFValidationError: ignored